Training Deepforest model on data https://github.com/jonathanventura/urban-tree-detection-data.git and testing it respectively

In [ ]:
#@title Installing libraries for training and testing of data
!pip install --upgrade deepforest albumentations pyyaml
#pip install git+https://github.com/weecology/DeepForest.git
!pip uninstall opencv-python-headless -y
!pip install opencv-python-headless==4.1.2.30
!pip install GDAL
!pip install matplotlib
!pip uninstall pytorch_lightning -y
!pip install pytorch_lightning

In [ ]:
#@title fetching data for training and testing
!git clone https://github.com/jonathanventura/urban-tree-detection-data.git

In [1]:
#@title Required libraries to import
from pytorch_lightning.loops import dataloader
import pandas as pd 
import torch
from torch.utils.data import Dataset, DataLoader
import deepforest
from deepforest import main
import pytorch_lightning as pl
import os
import rasterio
import glob

In [ ]:
#@title Training of model on our data
class Training:

  # using deepforest model
  model = main.deepforest()

  def training_of_model(self, testing_csv, image_path):
    os.makedirs('data',exist_ok=True)

    annotationFileTemp=testing_csv

    # making data feasible to train on our model
    df = pd.read_csv(annotationFileTemp) 
    df['image_path'] = image_path
    df['xmin']=df['x'] - 1
    df['xmax']=df['x'] + 1
    df['ymin']=df['y'] - 1
    df['ymax']=df['y'] + 1
    df['label']="Tree"
    df.to_csv("/content/data/annotation.csv")

    # model configuration
    annotationFile="/content/data/annotation.csv"

    self.model.config["gpus"] = "-1"
    self.model.config["train"]["epochs"] = 5
    self.model.config["epochs"] = 5
    self.model.config["train"]["csv_file"] = annotationFile
    self.model.config["score_thresh"] = 0.4
    self.model.config["train"]["root_dir"] = os.path.dirname(annotationFile)
    self.model.config["train"]["fast_dev_run"] = False

    # model.create_trainer()
    # model.use_release()
    # model.trainer.fit(model)

    # creating trainer to train the model
    trainer= pl.Trainer( max_epochs=5,
                gpus="-1",
                enable_checkpointing=False,
                accelerator='gpu',
                fast_dev_run= False)
    
    # training model on the respective data
    trainer.fit(model=self.model)

In [5]:
#@title Total files in our data
# Counting training files in our data
counter=0
for file in glob.iglob(f"/content/urban-tree-detection-data/csv/*"):
  counter+=1
counter

571

In [ ]:
#@title Proving data to our model
training=Training()
counter=0

# Giving our model respective data files to train
for file in glob.iglob(f"/content/urban-tree-detection-data/csv/*"):
  if counter<100:
    file_name=""
    try:
      file_name=file
      file_name=f"/content/urban-tree-detection-data/images/{file[39:-4]}.tif"
      training.training_of_model(testing_csv=file, image_path=file_name)
    except:
        print(f"File has some invalid data, Filename : {file}")

In [26]:
#@title Testing our model
os.makedirs('deepforest',exist_ok=True)
training=Training()
names = [name.rstrip() for name in open('/content/urban-tree-detection-data/test.txt','r')]
for name in names:  
    image_path = os.path.join('/content/urban-tree-detection-data/images',f'{name}.tif')
    output_path = os.path.join('deepforest',f'{name}.json')
    with rasterio.open(image_path,'r') as src:
        transform = src.transform   
        crs = src.crs
    pred = training.model.predict_image(path=image_path)
    gdf = deepforest.utilities.annotations_to_shapefile(pred, transform, crs)
    gdf.to_file(output_path,driver='GeoJSON')